# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

## Задание 1. Напишите свою функцию prefilter_items

In [1]:
def prefilter_items(data, item_features=None, take_n_popular=5000, 
                    filters={'top_popular': 0.5, 
                             'top_notpopular': 0.01,
                             'price_less_than': 1,
                             'price_higher_than': 30,
                             'last_purchase_n_weeks_back': 50,
                             'department_filter': []
                            }):
    """Предфильтрация товаров
    
    Input
    ------
    data: pd.DataFrame, 
          Содержит столбцы 'item_id', 'user_id', 'sales_value', 'quantity', 'week_no'
          
    item_features: pd.DataFrame,
                   Содержит фичи товаров. Необходим для фильтрации по категориям. 
    
    take_n_popular: int 
                    Количество возвращаемых товаров
    filters: dict
             словарь, в котором ключами являются названия фильтров, которые необходимо применить, а значениями пороги, 
             по которым необходимо фильтровать. По умолчанию применяются все фильтры, кроме 'department_filter'
             Возможные значения: 'top_popular', 'top_notpopular', 'price_less_than', 'price_higher_than',
                               'last_purchase_n_weeks_back', 'department_filter'
    
    Return
    ------
    data - dataframe, отфильтрованнный в соответствии с указанными filters и содержащий топ-take_n_popular товаров
    """
    # Проверяем, что фильтры указаны верно
    default_filters = set(['top_popular', 'top_notpopular', 'price_less_than', 'price_higher_than', 
                          'last_purchase_n_weeks_back', 'department_filter'])
    
    assert all(filter_ in default_filters for filter_ in filters.keys()), ('Invalid filters\' keys. '
                                                                           'Please check help for '
                                                                           'getting valid values')
    
    # 1. Неинтересные категории
    if filters.get('department_filter'):
        assert item_features is not None, 'Function expects \'item_features\' dataframe for department filter'
        
        # Проверяем, что все фильтруемые категории присутствуют в датасете
        department_mask = [department in set(item_features['department'].unique()) 
                           for department in filters.get('department_filter')]
        not_exist = [filters.get('department_filter')[i] for i in range(len(department_mask))
                    if not department_mask[i]]
        
        assert all(department_mask), 'Following departments in department_filter do not exist in '\
                                    f'item_features dataframe {not_exist}'
        
        # Фильтруем по категориям
        from_filtered_department = item_features[item_features['department'].\
                                                 isin(filters.get('department_filter'))].item_id.tolist()
        data = data[~data['item_id'].isin(from_filtered_department)]
    
    # Фильтрация из урока
    # Подготовка для фильтрации по популярности
    if filters.get('top_popular') is not None or filters.get('top_notpopular') is not None:
        popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data['user_id'].nunique()
        popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
        
    # Самые популярные (итак купят)
    if filters.get('top_popular') is not None:
        top_popular = popularity[popularity['share_unique_users'] > filters.get('top_popular')].item_id.tolist()
        data = data[~data['item_id'].isin(top_popular)]
    
    # Самые непопулярные (итак не купят)
    if filters.get('top_notpopular') is not None:
        top_notpopular = popularity[popularity['share_unique_users'] < filters.get('top_notpopular')].item_id.tolist()
        data = data[~data['item_id'].isin(top_notpopular)]
    
    # Подготовка для фильтрации по цене
    if filters.get('price_less_than') is not None or filters.get('price_higher_than') is not None:
        data['price'] = data['sales_value'] / data['quantity']
        mean_prices = data.groupby('item_id')['price'].agg('mean').reset_index()
        mean_prices.rename(columns={'price': 'mean_price'}, inplace=True)
        
    # 2. Удаление товаров, со средней ценой < 1$
    if filters.get('price_less_than') is not None:
        low_price = mean_prices[mean_prices['mean_price'] < filters.get('price_less_than')].item_id.tolist()
        data = data[~data['item_id'].isin(low_price)]
    
    # 3. Удаление товаров со соедней ценой > 30$
    if filters.get('price_higher_than') is not None:
        high_price = mean_prices[mean_prices['mean_price'] > filters.get('price_higher_than')].item_id.tolist()
        data = data[~data['item_id'].isin(high_price)]
    
    # 4. Товары, которые не продавались 50 недель
    if filters.get('last_purchase_n_weeks_back') is not None:
        n_weeks_back = data['week_no'].unique().max() - filters.get('last_purchase_n_weeks_back')
        item_ids_before = set(data[data['week_no'] < n_weeks_back].item_id.tolist())
        item_ids_after = set(data[data['week_no'] >= n_weeks_back].item_id.tolist())
        item_ids_last_purchase_n_weeks_back = list(item_ids_before - item_ids_after)
        data = data[~data['item_id'].isin(item_ids_last_purchase_n_weeks_back)]
    
    # 5. Выбор топ-N самых популярных товаров (N = take_n_popular)
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
    top_n = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()
    
    data = data[data['item_id'].isin(top_n)]
    
    return data

В корне проекта создайте модуль *src*. Положите функцию *prefilter_items* в файл *src/utils.py*  

Далее запустите код ниже и перейдите к заданию 2 (в конце ноутбука)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [3]:
data = pd.read_csv('../raw_data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Warm start
train_items = data_train['item_id'].unique()
train_users = data_train['user_id'].unique()

data_test = data_test[data_test['item_id'].isin(train_items)]
data_test = data_test[data_test['user_id'].isin(train_users)]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
item_features = pd.read_csv('../raw_data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [6]:
n_items_before = data_train['item_id'].nunique()

# Пример полного вызова функции
# data_train = prefilter_items(data_train, item_features, take_n_popular=5000, 
#                              filters={'top_popular': 0.5, 
#                              'top_notpopular': 0.01,
#                              'price_less_than': 1,
#                              'price_higher_than': 30,
#                              'last_purchase_n_weeks_back': 50,
#                              'department_filter': ['AUTOMOTIVE']})

data_train = prefilter_items(data_train, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5000


In [7]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,818981,819063,819112,819255,819304,819308,819330,819518,819765,819840,...,15717057,15741861,15742123,15831255,15926712,15926775,15926844,15926886,15926927,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [9]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [10]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


CPU times: user 3.93 s, sys: 46.3 ms, total: 3.98 s
Wall time: 1.05 s


-----

# Домашнее задание

1. Перенесите метрики из ДЗ 1 src/metrics.py
3. Создайте модуль src/recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src/recommenders.py
4. Проверьте, что все модули корректно импортируются

In [11]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, item_features, weighting=True):
                
        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != 999999]

        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['quantity'].count().reset_index()
        self.overall_top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.overall_top_purchases = self.overall_top_purchases[self.overall_top_purchases['item_id'] != 999999]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()
        
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid,\
            self.itemid_to_id, self.userid_to_id = prepare_dicts(self.user_item_matrix)
        self.sparse_user_item = csr_matrix(user_item_matrix).tocsr()
        
        # Словарь {item_id: 0/1}. 0/1 - факт принадлежности товара к СТМ
        self.item_id_to_ctm = self.prepare_item_id_to_ctm(item_features)
        
        # Own recommender обучается до взвешивания матрицы
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        
    def get_recommendations(self, user, N=5):
        """Рекомендуем топ-N товаров"""
    
        res = [self.id_to_itemid[rec[0]] for rec in 
                    self.model.recommend(userid=self.userid_to_id[user], 
                                         user_items=self.sparse_user_item,   # на вход user-item matrix
                                         N=N, 
                                         filter_already_liked_items=False, 
                                         filter_items=[itemid_to_id[999999]],  # !!! 
                                         recalculate_user=True)]
        return res

    def get_similar_items_recommendation(self, user, filter_ctm=True, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        # your_code
        # Практически полностью реализовали на прошлом вебинаре
        # Не забывайте, что нужно учесть параметр filter_ctm
        top_n_user_purchases = self.top_purchases[self.top_purchases['user_id'] == user].head(N).item_id.tolist()
        res = []
        
        if filter_ctm:
            # N=50, чтобы повысить шанс найти среди похожих товаров найти товар ctm
            for item_id in top_n_user_purchases:
                similar_items = self.model.similar_items(itemid_to_id[item_id], N=50)
                for sim_item in similar_items:
                    if self.item_id_to_ctm[id_to_itemid[sim_item[0]]] == 1:
                        res.append(sim_item[0])
                        break # Останавливаем цикл после 1го найденного товара и переходим к следующей покупке юзера
        
        else:
            for item_id in top_n_user_purchases:
                similar_item = self.model.similar_items(itemid_to_id[item_id], N=2)
                res.append(similar_item[1][0])
                
        res = [id_to_itemid[item] for item in res]
        return res
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
        
        similar_users = self.model.similar_users(self.userid_to_id[user], N=N)
        similar_users = [self.id_to_userid[user[0]] for user in similar_users]
        
        similar_users_items = self.top_purchases[self.top_purchases['user_id'].isin(similar_users)]
        similar_users_items.sort_values('quantity', ascending=False, inplace=True)
        
        res = similar_users_items['item_id'].head(N).tolist()
        return res
    
    @staticmethod
    def prepare_matrix(data):
        """Подготавливает user-item матрицу"""
        
        user_item_matrix = pd.pivot_table(data, 
                                          index='user_id', columns='item_id', 
                                          values='quantity', # Можно пробоват ьдругие варианты
                                          aggfunc='count', 
                                          fill_value=0
                                         )

        user_item_matrix = user_item_matrix.astype(float)
        
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
    
    @staticmethod
    def prepare_item_id_to_ctm(item_features):
        all_ids = item_features['item_id'].tolist()
        ctm_ids = set(item_features[item_features['brand'] == 'Private'].item_id.tolist())
        
        item_id_to_ctm = {}
        
        for item_id in all_ids:
            item_id_to_ctm[item_id] = 1 if item_id in ctm_ids else 0
            
        return item_id_to_ctm
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads)
        model.fit(csr_matrix(self.user_item_matrix).T.tocsr())
        
        return model

----

Проверка, что все работает

In [12]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

